## Allele filp QC YML generator
This module takes in a table of sumstat, with the columns: #chr, theme1, theme2, theme3 and each rows as 1 chr and the sumstat of corresponding chr and generate a list of yml to be used

In [ ]:
sos run pipeline/yml_generator.ipynb yml_list \
    --sumstat-list output/TensorQTL.cis._recipe.tsv  \
    --cwd  output/data_intergration/TensorQTL --name MWE &

In [1]:
[global]
# List of path to the index of sumstat, each correspond to 1 recipe file documenting the path to the sumstat of each chromosome.
parameter: sumstat_list = paths
# List of names that corresponding to each of the studies
parameter: name = list
parameter: cwd = path("output")
import pandas as pd
input_list = sumstat_list
sumstat_list = pd.read_csv(input_list[0],sep = "\t")
sumstat_list = sumstat_list.sort_values('#chr').reset_index(drop=True)
if len(input_list) > 1:
    for x in range(1,len(input_list)):
        sumstat_list = sumstat_list.merge(pd.read_csv(input_list[x],sep = "\t"), on = "#chr")
sumstat_meta = sumstat_list.filter(regex='column').iloc[0].values.tolist()
sumstat_list = sumstat_list.drop(sumstat_list.filter(regex='column').columns,axis = 1)
sumstat_list.columns = ["#chr"] + name
sumstat_inv = sumstat_list.values.tolist()
names = "_".join(name)
parameter: container = ""

In [ ]:
[yml_generator]
input: for_each  = "sumstat_inv"
output: f'{cwd:a}/{names}.{_sumstat_inv[0]}/{names}.{_sumstat_inv[0]}.yml'
python: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout' , container = container
    import os
    import yaml
    import pandas as pd
    output = dict()
    ## Input dict
    output["INPUT"] = [pd.read_csv(y,"\t",index_col = 0,names = [x],header = 0 ).to_dict() for x,y in zip($[_sumstat_inv[1:len(_sumstat_inv)]],$[sumstat_meta])]       
    ## Output dict
    output["OUTPUT"] = [$[_output:dr]]
    with open($[_output:ar], 'w') as f:
        yaml.dump(output,f,sort_keys = False)

In [ ]:
[yml_list]
input: output_from("yml_generator"),group_by = "all"
output: f'{cwd:a}/yml_list.txt', f'{cwd:a}/qced_sumstat_list.txt'
import pandas as pd
yml_df = pd.DataFrame({"#chr" : sumstat_list["#chr"].values.tolist()  ,  "dir" : _input})
yml_df.to_csv(_output[0],sep = "\t",index = 0)
data_dir_tmp = pd.Series(_input)
data_dir = [f'{x:d}' for x in data_dir_tmp ]
theme = sumstat_list.columns.values.tolist()[1:]
for i in theme:
    sumstat_list = sumstat_list.assign(**{i : data_dir+ pd.Series([f'/{path(x):b}'  for x in  sumstat_list[i].values.tolist()])  } )
sumstat_list.to_csv(_output[1],sep = "\t", index = 0)